In [1]:
%run "Setup_Env.ipynb"

## Mini-Project 4 - IT Support Analyst

Ask ChatGPT to act as a IT support agent, process each customer IT ticket message and output the response in JSON with the following fields

```
orig_msg: The original customer message
orig_lang: Detected language of the customer message e.g. Spanish
category: 1-2 word describing the category of the problem
trans_msg: Translated customer message in English
response: Response to the customer in orig_lang
trans_response: Response to the customer in English
```

Try to use a JSON parser to get the responses in JSON for each ticket

In [2]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field


# Define your desired data structure - like a python data class.
class ITSupportResponse(BaseModel):
    orig_msg: str = Field(description="The original customer IT support query message")
    orig_lang: str = Field(description="Detected language of the customer message e.g. Spanish")
    category: str = Field(description="1-2 word describing the category of the problem")
    trans_msg: str = Field(description="Translated customer IT support query message in English")
    response: str = Field(description="Response to the customer in their original language - orig_lang")
    trans_response: str = Field(description="Response to the customer in English")


parser = JsonOutputParser(pydantic_object=ITSupportResponse)

/Users/sourav.banerjee/Documents/My Codebases/GenerativAI_Demystified/.venv/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3699: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
# create the final prompt with formatting instructions from the parser
prompt_txt = """
             Act as an Information Technology (IT) customer support agent.
             For the IT support message mentioned below
             use the following output format when generating the output response

             Output format instructions:
             {format_instructions}

             Customer IT support message:
             {it_support_msg}
             """
prompt = PromptTemplate(
    template=prompt_txt,
    input_variables=["it_support_msg"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

In [4]:
# create a simple LCEL chain to take the prompt, pass it to the LLM, enforce response format using the parser
llm_chain = (prompt
              |
            chatgpt
              |
            parser)

In [5]:
it_support_queue = [
    "Não consigo sincronizar meus contatos com o telefone. Sempre recebo uma mensagem de falha.",
    "Ho problemi a stampare i documenti da remoto. Il lavoro non viene inviato alla stampante di rete.",
    "プリンターのトナーを交換しましたが、印刷品質が低下しています。サポートが必要です。",
    "Я не могу войти в систему учета времени, появляется сообщение об ошибке. Мне нужна помощь.",
    "Internet bağlantım çok yavaş ve bazen tamamen kesiliyor. Yardım eder misiniz?",
    "Не могу установить обновление безопасности. Появляется код ошибки. Помогите, пожалуйста."
]

formatted_msgs = [{"it_support_msg": msg}
                    for msg in it_support_queue]
formatted_msgs[0]

{'it_support_msg': 'Não consigo sincronizar meus contatos com o telefone. Sempre recebo uma mensagem de falha.'}

In [6]:
responses = llm_chain.map().invoke(formatted_msgs)

In [7]:
responses[0]

{'orig_msg': 'Não consigo sincronizar meus contatos com o telefone. Sempre recebo uma mensagem de falha.',
 'orig_lang': 'Portuguese',
 'category': 'Sync Issue',
 'trans_msg': "I can't sync my contacts with the phone. I always receive a failure message.",
 'response': 'Por favor, verifique se você está conectado à internet e tente reiniciar o telefone. Se o problema persistir, considere desinstalar e reinstalar o aplicativo de contatos.',
 'trans_response': 'Please check if you are connected to the internet and try restarting the phone. If the problem persists, consider uninstalling and reinstalling the contacts app.'}

In [11]:
print(responses[0]["orig_msg"])
print(responses[0]["orig_lang"])
print(responses[0]["category"])
print(responses[0]["trans_msg"])
print(responses[0]["response"])
print(responses[0]["trans_response"])

Não consigo sincronizar meus contatos com o telefone. Sempre recebo uma mensagem de falha.
Portuguese
Sync Issue
I can't sync my contacts with the phone. I always receive a failure message.
Por favor, verifique se você está conectado à internet e tente reiniciar o telefone. Se o problema persistir, considere desinstalar e reinstalar o aplicativo de contatos.
Please check if you are connected to the internet and try restarting the phone. If the problem persists, consider uninstalling and reinstalling the contacts app.


In [8]:
import pandas as pd

df = pd.DataFrame(responses)
df

,orig_msg,orig_lang,category,trans_msg,response,trans_response
0,Não consigo sincronizar meus contatos com o te...,Portuguese,Sync Issue,I can't sync my contacts with the phone. I alw...,"Por favor, verifique se você está conectado à ...",Please check if you are connected to the inter...
1,Ho problemi a stampare i documenti da remoto. ...,Italian,Printing Issue,I have problems printing documents remotely. T...,Mi dispiace sapere che hai problemi a stampare...,I'm sorry to hear that you're having trouble p...
2,プリンターのトナーを交換しましたが、印刷品質が低下しています。サポートが必要です。,Japanese,Printer Issue,"I replaced the printer toner, but the print qu...",トナーを交換した後に印刷品質が低下することがあります。まず、トナーが正しく取り付けられている...,It is possible for print quality to decrease a...
3,"Я не могу войти в систему учета времени, появл...",Russian,Login Issue,"I cannot log into the time tracking system, an...","Пожалуйста, проверьте свои учетные данные и уб...",Please check your credentials and ensure you a...
4,Internet bağlantım çok yavaş ve bazen tamamen ...,Turkish,Internet Issue,My internet connection is very slow and someti...,Bağlantı sorunlarınızı çözmek için birkaç adım...,We can take a few steps to resolve your connec...
5,Не могу установить обновление безопасности. По...,Russian,Update Issue,I cannot install the security update. An error...,"Пожалуйста, уточните, какой код ошибки появляе...",Please specify what error code appears when in...
